In [11]:
!pip install lxml
!pip install bs4
!pip install html5lib

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=f2edf4ebf0e9f7056a625337a0041ce4863d01b4d0139935638cf62d49536c35
  Stored in directory: /home/jovyan/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
     |████████████████████████████████| 112 kB 13.7 MB/s eta 0:00:01


In [12]:
from bs4 import BeautifulSoup  
import pandas as pd
from tqdm import tqdm
import math
import requests  
import lxml
import re
import time

In [13]:
area_dic = {
    '静安区':'jingan',
    '徐汇区':'xuhui',
    '黄埔区':'huangpu',
    '长宁区':'changning',
    '普陀区':'putuo',
    '浦东新区':'pudong',
    '宝山区':'baoshan',
    '虹口区':'hongkou',
    '杨浦区':'yangpu',
    '闵行区':'minhang',
    '金山区':'jinshan',
    '嘉定区':'jiading',
    '崇明区':'chongming',
    '奉贤':'fengxian',
    '松江':'songjiang',
    '青浦':'qingpu'
            }

In [14]:
# 加个header以示尊敬
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
           'Referer': 'https://sh.lianjia.com/zufang/'}

# 新建一个会话
sess = requests.session()
sess.get('https://sh.lianjia.com/zufang/', headers=headers)

# url示例：https://sz.lianjia.com/zufang/luohuqu/pg2/
url = 'https://sh.lianjia.com/zufang/{}/pg{}/'

In [15]:
# 当正则表达式匹配失败时，返回默认值（errif）
def re_match(re_pattern, string, errif=None):
    try:
        return re.findall(re_pattern, string)[0].strip()
    except IndexError:
        return errif

In [17]:
# 新建一个DataFrame存储信息
data = pd.DataFrame()

for key_, value_ in area_dic.items():
    # 获取该行政区下房源记录数
    start_url = 'https://sh.lianjia.com/zufang/{}/'.format(value_)
    html = sess.get(start_url).text
    house_num = re.findall('已为您找到 <span class="content__title--hl">(.*?)</span>', html)[0].strip()
    print('🏠{}: 出租房源共计「{}」套'.format(key_, house_num))
    time.sleep(1)
    # 页面限制 每个行政区只能获取最多100页共计3000条房源信息
    total_page = int(math.ceil(min(3000, int(house_num)) / 30.0))
    for i in tqdm(range(total_page), desc=key_):
        html = sess.get(url.format(value_, i+1)).text
        soup = BeautifulSoup(html, 'html.parser')
        info_collect = soup.find_all(class_="content__list--item")
        
        for info in info_collect:
            info_dic = {}
            # 行政区
            info_dic['area'] = key_

            # 房源的标题
            info_dic['title'] = re_match('title="(.*?)"', str(info))
            # 位置
            info_dic['position'] = re_match('</a>-<a.*?target="_blank">(.*?)</a>', str(info))
            #小区
            info_dic['community'] = re_match('</a>-<a.*?target="_blank">.*?</a>.*?title="(.*?)"', str(info))
            # 总价
            info_dic['total_price'] = float(re_match('class="content__list--item-price"><em>(.*?)</em> 元/月</span>', str(info)))
            #面积
            info_dic['size'] = re_match('<i>/</i>\n(.*?)㎡', str(info))

                
            # 存入DataFrame
            if data.empty:
                data = pd.DataFrame(info_dic,index=[0])
            else:
                data = data.append(info_dic,ignore_index=True)

🏠静安区: 出租房源共计「2276」套


静安区: 100%|██████████| 76/76 [04:31<00:00,  3.57s/it]


🏠徐汇区: 出租房源共计「2181」套


徐汇区: 100%|██████████| 73/73 [04:08<00:00,  3.40s/it]


🏠黄埔区: 出租房源共计「1179」套


...

奉贤: 100%|██████████| 17/17 [00:53<00:00,  3.15s/it]


🏠松江: 出租房源共计「1675」套


松江: 100%|██████████| 56/56 [02:38<00:00,  2.83s/it]


🏠青浦: 出租房源共计「1298」套


青浦: 100%|██████████| 44/44 [02:14<00:00,  3.06s/it]


In [18]:
data.shape

(23629, 6)

In [19]:
data.sample(20)

,area,title,position,community,total_price,size
17545,闵行区,整租·东亚梧桐公馆 4室2厅 南,颛桥,东亚梧桐公馆,30000.0,582.54
20931,松江,整租·华亭新家百合苑 2室1厅 南/北,松江老城,华亭新家百合苑,3200.0,60.22
5902,长宁区,整租·森晟世洋国际广场 1室1厅 南,西郊,森晟世洋国际广场,4500.0,33.70
12732,宝山区,整租·央玺 3室2厅 南,共富,央玺,8200.0,84.53
6830,长宁区,整租·南辉小区 1室1厅 南,中山公园,南辉小区,6800.0,35.83
2826,徐汇区,整租·尚海湾豪庭(二期） 2室2厅 南,徐汇滨江,尚海湾豪庭(二期）,38000.0,119.37
1644,静安区,整租·卓悦居 2室1厅 南/北,不夜城,卓悦居,14000.0,93.47
19382,嘉定区,整租·新城金郡（商住） 2室2厅 南,嘉定新城,新城金郡（商住）,3200.0,39.76
20313,奉贤,整租·绿地香颂(奉贤)(公寓) 1室1厅 南,海湾,绿地香颂(奉贤)(公寓),1900.0,62.37
9699,浦东新区,整租·泾东小区 1室1厅 南,洋泾,泾东小区,5200.0,41.63


In [20]:
!pip install openpyxl

In [21]:
data.to_csv('lianjia_house_leasing_22.csv',index = 0)